In [ ]:
import mne 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

### Load Data

In [ ]:
raw = mne.io.read_raw_eeglab('rEEG\sub-001\eeg\sub-001_task-Rest_eeg.set',
                              preload=True)
raw.info  # check channels, sampling rate, etc.
print(raw.ch_names)
 


### Mark Bad Channels (dataset specified)

In [ ]:
# After visual inspection, mark bad channels
raw.info['bads'] = [raw.ch_names[45], raw.ch_names[4], raw.ch_names[47]]
raw.plot(scalings='auto', block=True, title='Raw Data with Bad Channels Marked')

### Set montage


In [ ]:
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

### Autodetect additional bad channels

In [ ]:
from mne.preprocessing import find_bad_channels_lof
# Find bad channels using LOF
bads_lof = find_bad_channels_lof(raw, n_neighbors=13, threshold=1.5, picks='eeg')
raw.info['bads'].extend(bads_lof) # Add detected bads to raw.info['bads']



### Interpolate bad channels

In [ ]:
raw.interpolate_bads(reset_bads=True)

### Re-reference to common average

In [ ]:
raw.set_eeg_reference('average')
raw.plot(scalings='auto', block=True, title='Rereferenced Data')

### Band-pass filter (1-45 Hz)

In [ ]:
raw.filter(1., 45., fir_design='firwin')
raw.plot(scalings='auto', block=True, title='Band Pass Filtered Data')

### Downsample to 250 Hz

In [ ]:

raw.resample(250, npad="auto")
raw.plot(scalings='auto', block=True, title='Resampled Data')

### Plot PSD and cleaned signal

In [ ]:
raw.plot(scalings='auto', block=True, title='Preprocessed Data Before ICA')
raw.plot_psd(fmax=60)

# SAVE DATA PER SUBJECT BEFORE ICA INTO THEIR FOLDER

In [ ]:
raw.save('rEEG/sub-001/preICA_raw.fif', overwrite=True)


## ICA (Independent Component Analysis) & Artifact Removal

### Run ICA

In [ ]:
from mne.preprocessing import ICA

ica = ICA(n_components= 20, random_state=97, max_iter='auto')
ica.fit(raw)

ica.plot_components()



### Reject ICA components

In [68]:
# show component topographies
ica.plot_components(picks=[0,3,6,13])

# for each candidate, check properties: time series + PSD + sources
for comp in [0,3,6,13]:
    ica.plot_properties(raw, picks=comp)


KeyboardInterrupt: 

In [ ]:
ica.exclude = []
raw_clean = ica.apply(raw.copy())



### Apply ICA to raw

In [ ]:
raw_clean.plot(scalings = 'auto',title='After ICA', block=True)
raw_clean.plot_psd(fmax=60)